In [2]:
import pandas as pd

movies_df = pd.read_csv('ml-latest-small/movies.csv')

# Extract unique genres
# Split genres by '|' and get all unique genres
unique_genres = set()
for genres in movies_df['genres']:
    unique_genres.update(genres.split('|'))
unique_genres = sorted(unique_genres)  # Sort in alphabetic order

# Genre vector
for genre in unique_genres:
    movies_df[genre] = movies_df['genres'].apply(lambda x: 1 if genre in x.split('|') else 0)

# Drop the original 'genres' column (optional)
movies_df = movies_df.drop(columns=['genres'])

print(movies_df.head())


   movieId                               title  (no genres listed)  Action  \
0        1                    Toy Story (1995)                   0       0   
1        2                      Jumanji (1995)                   0       0   
2        3             Grumpier Old Men (1995)                   0       0   
3        4            Waiting to Exhale (1995)                   0       0   
4        5  Father of the Bride Part II (1995)                   0       0   

   Adventure  Animation  Children  Comedy  Crime  Documentary  ...  Film-Noir  \
0          1          1         1       1      0            0  ...          0   
1          1          0         1       0      0            0  ...          0   
2          0          0         0       1      0            0  ...          0   
3          0          0         0       1      0            0  ...          0   
4          0          0         0       1      0            0  ...          0   

   Horror  IMAX  Musical  Mystery  Romance  

In [3]:
from collections import Counter
tags_df = pd.read_csv('ml-latest-small/tags.csv')

unique_tag = set() # set use to get number of unique tags
count_tag = [] # working with arrays is easier, also all tags in the csv

for tags in tags_df['tag']:
    # This is the most common tag, but its not relevent to the movie so I skip it
    if (tags == 'In Netflix queue'):
        continue
    count_tag.append(tags)
    unique_tag.add(tags)

# get most common
count = Counter(count_tag)
max_tag = count.most_common(60)

name_of_tag = []
for x in max_tag:
    name_of_tag.append(x[0])

# number of unique tags
print(len(unique_tag))
# All 60 tags with their num uses 
print(max_tag)
# we get the top 60 tags
print(name_of_tag)

1588
[('atmospheric', 36), ('superhero', 24), ('thought-provoking', 24), ('funny', 23), ('Disney', 23), ('surreal', 23), ('religion', 22), ('sci-fi', 21), ('dark comedy', 21), ('quirky', 21), ('psychology', 21), ('suspense', 20), ('twist ending', 19), ('visually appealing', 19), ('crime', 19), ('politics', 18), ('music', 16), ('time travel', 16), ('mental illness', 16), ('dark', 15), ('comedy', 15), ('aliens', 15), ('space', 14), ('mindfuck', 14), ('dreamlike', 14), ('black comedy', 13), ('emotional', 13), ('heist', 13), ('anime', 12), ('action', 12), ('satire', 12), ('high school', 12), ('disturbing', 12), ('Shakespeare', 12), ('journalism', 12), ('Stephen King', 12), ('court', 12), ('imdb top 250', 11), ('comic book', 11), ('classic', 11), ('psychological', 11), ('Holocaust', 11), ('adolescence', 11), ('adultery', 11), ('boxing', 11), ('drugs', 10), ('Leonardo DiCaprio', 10), ('Mafia', 10), ('animation', 10), ('robots', 10), ('cinematography', 10), ('India', 10), ('ghosts', 10), ('re

In [ ]:
movies_df = pd.read_csv('ml-latest-small/movies.csv')
tags_set = tags_df.groupby(['movieId']).agg({'tag':set}).reset_index()
print(tags_set.head(10))

# Goes over the most common tags
# adds column tags and goes through movie tags using the tags_set,
# applies 1 if tag in tag_set  
for tag in name_of_tag:
    movies_df[tag] = tags_set['tag'].apply(lambda x: 1 if tag in x else 0)

    
print(movies_df.head(10))
# can't see the 1s being added using movies_df.head() so i made a file
movies_df.to_csv('movie_vct.csv',index=False)

   movieId                                                tag
0        1                                       {fun, pixar}
1        2  {game, Robin Williams, fantasy, magic board game}
2        3                                       {moldy, old}
3        5                                {remake, pregnancy}
4        7                                           {remake}
5       11                              {politics, president}
6       14                              {politics, president}
7       16                                            {Mafia}
8       17                                      {Jane Austen}
9       21                                        {Hollywood}
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   
5        6        